### Extracting BERT embeddings from texts for readability assessment.

In [ ]:
!pip install -U sentence-transformers

     |████████████████████████████████| 81kB 5.3MB/s 
     |████████████████████████████████| 2.3MB 23.1MB/s 
     |████████████████████████████████| 1.2MB 36.4MB/s 
     |████████████████████████████████| 3.3MB 50.9MB/s 
     |████████████████████████████████| 901kB 40.8MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-1.2.0-cp37-none-any.whl size=123339 sha256=dcfab51b03eeb25dda0f1aaccdf36b8459b29407a3af6c4e0173b72330f5ea60
  Stored in directory: /root/.cache/pip/wheels/0f/06/f7/faaa96fdda87462b4fd5c47b343340e9d5531ef70d0eef8242
Successfully built sentence-transformers


In [ ]:
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModel
import torch
import pickle
import numpy

Read contents from a specific corpus. Corpus should be 1 per line and in .txt format.

In [ ]:
titles = []
contents = []

with open("commoncore.txt","r") as file:
  file_contents = file.readlines()
  for i in file_contents:
    parsed_text = i.split(';',2)
    parsed_text[2] = parsed_text[2].strip()
    print(parsed_text[2])
    titles.append(parsed_text[0].strip())
    contents.append(parsed_text[2])

The policeman on the beat moved up the avenue impressively. The impressiveness was habitual and not for show, for spectators were few. The time was barely o'clock at night, but chilly gusts of wind with a taste of rain in them had well nigh de-peopled the streets. Trying doors as he went, twirling his club with many intricate and artful movements, turning now and then to cast his watchful eye adown the pacific thoroughfare, the officer, with his stalwart form and slight swagger, made a fine picture of a guardian of the peace. The vicinity was one that kept early hours. Now and then you might see the lights of a cigar store or of an all-night lunch counter but the majority of the doors belonged to business places that had long since been closed. When about midway of a certain block the policeman suddenly slowed his walk. In the doorway of a darkened hardware store a man leaned, with an unlighted cigar in his mouth. As the policeman walked up to him the man spoke up quickly. It's all rig

Mean Pooling - take attention mask into account for correct averaging.

In [ ]:
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    return sum_embeddings / sum_mask

Load BERT models from Huggingface model repository.

In [ ]:
# FOR FILIPINO
# tokenizer = AutoTokenizer.from_pretrained("jcblaise/bert-tagalog-base-cased")
# model = AutoModel.from_pretrained("jcblaise/bert-tagalog-base-cased")

# FOR ENGLISH
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
model = AutoModel.from_pretrained("bert-base-cased")

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Preprocess and compute embeddings

In [ ]:
#Tokenize sentences
encoded_input = tokenizer(contents, padding=True, truncation=True, max_length=512, return_tensors='pt')

#Compute token embeddings
with torch.no_grad():
    model_output = model(**encoded_input)

#Perform pooling. In this case, mean pooling
sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

Transform embeddings to numpy format. Show example at index 0.

In [ ]:
sentence_embeddings_np = sentence_embeddings.numpy()
sentence_embeddings_np[0]

array([ 3.00258428e-01, -2.55088925e-01, -4.44831848e-02,  2.43792981e-01,
        5.13282061e-01,  3.49847414e-02, -2.19882876e-01,  3.26985121e-02,
        8.45087543e-02,  7.38475844e-02,  6.68110251e-02,  3.10143769e-01,
       -1.87994301e-01,  2.98425049e-01, -1.29232749e-01, -1.97815269e-01,
        4.51567806e-02, -2.10220993e-01, -1.81306884e-01,  2.79641468e-02,
        1.04797877e-01, -4.71706912e-02, -1.86185449e-01,  9.06723365e-02,
        2.26074576e-01, -7.64455721e-02, -7.51836225e-02,  8.26824248e-01,
        8.53071362e-03,  2.60233492e-01, -1.04036227e-01, -2.09600888e-02,
       -9.93568450e-02, -2.24021181e-01,  1.23696715e-01, -9.36410874e-02,
       -1.32401615e-01,  3.53948295e-01, -5.06256409e-02,  1.63166493e-01,
        2.24918634e-01, -3.01274844e-03, -4.57667373e-02,  4.43920732e-01,
       -7.43953660e-02, -3.71251047e-01, -1.24648698e-01,  3.35631847e-01,
       -3.77991736e-01,  9.70294699e-02,  2.96499252e-01,  3.64546701e-02,
        1.38769373e-02, -

Save in csv format. Can be added to csv files of linguistic features for readability assessment.

In [ ]:
numpy.savetxt('commoncore_sbert_embeddings.csv', sentence_embeddings_np, delimiter=',')